<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

In [1]:
!conda install -c conda-forge beautifulsoup4 --yes

!conda install -c conda-forge geopy --yes

!conda install -c conda-forge folium=0.5.0 --yes

print('Libraries installed!')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries installed!


In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
from pandas.io.json import json_normalize
import json

import requests

from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print('Libraries imported!')

Libraries imported!


<h1>Part 1: Web scraping for Toronto neighborhood and build a clean dataframe</h1>

<p>Use the Notebook to build the code to scrape the following Wikipedia page, <a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M">https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M</a>, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.
<ul TYPE=disk>
    <li>The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood</li>
    <li>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.</li>
    <li>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will         notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined         into one row with the neighborhoods separated with a comma.</li>
    <li>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for           the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's           Park.</li>
    <li>Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.</li>
    <li>In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.</li>
</ul>
<b>Note:</b> There are different website scraping libraries and packages in Python. One of the most common packages is BeautifulSoup. Here is the package's main documentation page: <a href="http://beautiful-soup-4.readthedocs.io/en/latest/">http://beautiful-soup-4.readthedocs.io/en/latest/</a></p>

In [4]:
# open Wiki page with Beautiful Soup
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')

<b>Scrape the HTML</b>
<p>The data we want is in a table, with 3 columns <b><i>PostalCode, Borough and Neighborhood.</i></b>
The table contains a list of postal codes in Canada where the first letter is M. Postal codes beginning with M are located within the city of Toronto in the province of Ontario.
Only the first three characters are listed, corresponding to the Forward Sortation Area.<p>

In [5]:
postalCodeList = []
boroughList = []
neighborhoodList = []

for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) # remove the new line char from neighborhood cell

<b>Put the scraped data into dataframe</b>
<p>After scraping the html, we get 3 lists containing the PostalCodes, Boroughs and Neighborhoods in order as in Wiki page.
We will put those 3 lists into a dataframe with corresponding columns name.</p>

In [6]:
toronto_neighorhood = [('PostalCode', postalCodeList),
                      ('Borough', boroughList),
                      ('Neighborhood', neighborhoodList)]
toronto_df = pd.DataFrame.from_dict(dict(toronto_neighorhood))
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


<b>Remove not assigned rows</b> There are some postal codes which are not belongs to any borough.
We will remove those rows.

In [7]:
toronto_df_dropna = toronto_df[toronto_df.Borough != 'Not assigned'].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


<b>Group neighborhoods by postal and borough</b>
<p>There are some neighborhoods that belongs to the same postal code and borough (Ex: M5A Downtown Toronto has 2 neighborhoods Harbourfront and Regent Park).
We will concat them into same row, seperated by a colon.<p>

In [8]:
toronto_df_grouped = toronto_df_dropna.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<b>Deal with Not assigned Neighborhood</b>
<p>For M7A Queen's Park, there is no neighborhood assigned.
We will replace the 'Not assigned' with the value of the corresponding Borough<p>

In [9]:
na_neigh_rows = toronto_df_grouped.Neighborhood == 'Not assigned'
toronto_df_grouped.loc[na_neigh_rows, 'Neighborhood'] = toronto_df_grouped.loc[na_neigh_rows, 'Borough']
toronto_df_grouped[na_neigh_rows]

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


<b>DateFrame cleaned</b>
<p>The final dataframe has 103 rows</p>

In [16]:
toronto_df_cleaned = toronto_df_grouped
toronto_df_cleaned.shape

(103, 3)

<h1>Part 2: Getting coordinates and add to the Toronto DataFrame</h1>
<p>Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.</p>

<p>In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API:<a href="http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/"> http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/,</a> so we will use the Geocoder Python package instead:<a href="https://geocoder.readthedocs.io/index.html">https://geocoder.readthedocs.io/index.html.</a></p>

<p>The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates.</p>

<p>geolocator = Nominatim(user_agent="tl-toronto-neigh")</p>

<p>postalList = toronto_df_cleaned['PostalCode'].values latList = [] longList = []</p>

<p>for post in postalList: location = None while(location is None): location = geolocator.geocode('{}, Toronto, Ontario'.format(post)) if(location != None): lat = location.latitude long = location.longitude print(post, lat, long) latList.append(lat) longList.append(long)</p>

<p>toronto_coors = [('Postal Code', postalList), ('Latitude', latList), ('Longitude', longList)] coors = pd.DataFrame.from_items(toronto_coors)</p>

<p><b>Note:</b> Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code:<a href="http://cocl.us/Geospatial_data"> http://cocl.us/Geospatial_data</a></p>

<p>The csv file format has:</p>

<p>3 columns: Postal Code, Latitude and Longitude</p>
103 rows: corresponding to 103 postal codes in our toronto dataframe

In [11]:
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
print('Coordinates downloaded!')
coors = pd.read_csv('toronto_coordinates.csv')

Coordinates downloaded!


In [12]:
print(coors.shape)
coors.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We will merge the two dataframes.
To get the correct result independant on the order of data, we need to set indexes of two dataframes to its Postal Code columns

In [13]:
toronto_df_temp = toronto_df_cleaned.set_index('PostalCode')
coors_temp = coors.set_index('Postal Code')
toronto_df_coors = pd.concat([toronto_df_temp, coors_temp], axis=1, join='inner')

Reset index and we will get the toronto dataframe with coordinates

In [14]:
toronto_df_coors.index.name = 'PostalCode'
toronto_df_coors.reset_index(inplace=True)

Check the toronto dataframe with coordinates

In [15]:
print(toronto_df_coors.shape)
toronto_df_coors.head()

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
